## Agent Example

In [38]:
from langchain.agents import load_tools, AgentExecutor
from langchain_experimental.plan_and_execute import PlanAndExecute, load_chat_planner

from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.agents.format_scratchpad import format_log_to_messages
from langchain.tools.render import render_text_description
from langchain import hub

from chat_model_client import get_model
from pprint import pprint

## 示例代码

本示例的目的是让大模型在问答问题的过程中，可以自动调用搜索引擎和计算器来完成用的任务。

In [20]:
llm = get_model("deepseek")
llm_with_stop = llm.bind(stop=["\nObservation"])

tools = load_tools(["ddg-search", "llm-math"], llm=llm)

prompt = hub.pull("hwchase17/react")
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_messages(x["intermediate_steps"])
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

agnet_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agnet_executor.invoke({"input": "今天北京飞海南温度相差几度？从北京飞海南需要怎么穿衣？"})

/var/folders/5x/c0q41fpx6l540lsl42_bzk5h0000gq/T/ipykernel_36659/2448424658.py:1: UserWarning: Parameters {'frequency_penalty', 'top_p', 'presence_penalty'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  llm = get_model("openai")
/Users/shaon/anaconda3/envs/wrenv3.10/lib/python3.10/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
Action: duckduckgo_search
Action Input: "北京飞海南今天温度差"北京市气象台今晨6时发布天气预报，预计，今天白天晴间多云，北转南风二三级，最高气温6℃；夜间晴转多云，南转北风一二级，最低气温零下6℃。 周末两天，北京以晴到多云天气为主，气温变化不大，最高气温6℃，最低气温零下3℃至零下4℃。 中国天气网讯 今天（1月5日），北京天气多云间晴，山区有零星小雪，偏北风二三级间四级，最低气温仅-3℃，公众外出需注意防风防寒。 明后天，北京空气持续干燥，昼夜温差仍较大，需持续做好防寒工作，及时补水润燥。 海南天气预报，及时准确发布中央气象台天气信息，便捷查询北京今日天气，海南周末天气，海南一周天气预报，海南15日天气预报，海南40日天气预报，海南天气预报还提供海南各区县的生活指数、健康指数、交通指数、旅游指数，及时发布海南气象预警信号、各类气象资讯。 未来三天，北京天气将以晴到多云为主，多北风，后天将有强冷空气到访，带来大风降温天气。 北京市气象台预计，今天白天，北京晴转多云，高海拔山区零星小雪，偏北风3级左右，阵风6级左右，最高气温1℃。 今天(12月16日)，我国大部气温回升，但昼夜温差将拉大至10℃至15℃，公众请适时调整着装，谨防感冒。 17日之后，两轮冷空气影响我国，气温又将 ...Action: duckduckgo_search
Action Input: "北京飞海南今天温度差"中国天气网讯 今天（1月5日），北京天气多云间晴，山区有零星小雪，偏北风二三级间四级，最低气温仅-3℃，公众外出需注意防风防寒。 明后天，北京空气持续干燥，昼夜温差仍较大，需持续做好防寒工作，及时补水润燥。 海南天气预报，及时准确发布中央气象台天气信息，便捷查询北京今日天气，海南周末天气，海南一周天气预报，海南15日天气预报，海南40日天气预报，海南天气预报还提供海南各区县的生活指数、健康指数、交通指数、旅游指数，及时发布海南气象预警信号、各类气象资讯。 北京市气象台今晨6时发布天气预报，预计，今天白天晴间多云，北转南风二三级，最高气温6℃；夜间晴转多云，南转北风一二级，最低气温零下6℃。 周末两天，北京以晴到多云天气为主，气温变化不大，最高气温6℃，最低气温零下3

{'input': '今天北京飞海南温度相差几度？从北京飞海南需要怎么穿衣？',
 'output': 'The temperature difference between Beijing and Hainan today is 12 degrees Celsius. To dress appropriately when flying from Beijing to Hainan, it is recommended to wear warm clothing due to the cold temperatures in Beijing and the potential temperature drop in Hainan.'}

## 输出Graph流程

In [33]:
# prompt.invoke({"input": "{input}"})
# render_text_description(tools)
print(agent.get_prompts())
pprint(agent.get_graph().print_ascii())

[PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'duckduckgo_search - A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator(*args: Any, callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.', 'tool_names': 'duckduckgo_search, Calculator'}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should a